In [ ]:
!pip install langgraph

In [2]:
from typing import List, Dict, TypedDict
from pydantic import BaseModel, Field

In [3]:
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        keys: A dictionary where each key is a string.
    """

    keys: Dict[str, any]

In [4]:
greetings_pool = [["salve", "hello"],
 ["ciao", "hi"],
 ["bounaserra", "good evening"],
 ["buongiorno", "good morning"]]

In [5]:
def hello(state):
    print("----HELLO-----")
    state_dict = state["keys"]
    greetings = state_dict["greetings"]
    print(greetings)
    return {
        "keys": {"greetings": greetings}
    }

In [6]:
import random
def hello_back(state):
    print("------HELLO BACK---------")
    state_dict = state["keys"]
    choice = random.randint(0,3)
    return_greetings = greetings_pool[choice]
    print(return_greetings)
    return {
        "keys": {"greetings": greetings, "return_greetings":return_greetings}
    }

In [8]:
def greetings_grader(state):
    state_dict = state["keys"]
    greetings = state_dict["greetings"]
    return_greetings = state_dict["return_greetings"]
    grade = "good greeter"
    for greeting in return_greetings:
        if greeting in greetings:
            continue
        grade = "bad greeter"
        break
    if grade == "good greeter":
        print("DECISION-----GOOD GREETER----GOODBYE")
    else:
        print("DECISION-----BAD GREETER----GREET AGAIN")
    return grade

In [9]:
from langgraph.graph import StateGraph, END

# Define a new graph
workflow = StateGraph(GraphState)

# Define the two nodes we will cycle between
workflow.add_node("hello", hello)
workflow.add_node("hello_back", hello_back)

# Set the entrypoint as `hello`
workflow.set_entry_point("hello")

# We now add a conditional edge
# We grade the greetings from hello-back
# If grade is "good greeter", we end or we cycle
# back to hello-back
workflow.add_conditional_edges(

    "hello_back",
    greetings_grader,
    {
        "bad greeter": "hello_back",
        "good greeter": END
    }
)

workflow.add_edge('hello', 'hello_back')

app = workflow.compile()

type(app)

langgraph.pregel.Pregel

In [15]:
greetings = ["hello", "salve"]
try:
    app.invoke({"keys":{"greetings": greetings}}, config={"recursion_limit": 10})
except:
    print("Oh no, you reached the greeting limit!")

----HELLO-----
['hello', 'salve']
------HELLO BACK---------
['ciao', 'hi']
DECISION-----BAD GREETER----GREET AGAIN
------HELLO BACK---------
['salve', 'hello']
DECISION-----GOOD GREETER----GOODBYE
